In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import json
import os

import numpy as np
import pandas as pd
from openai import OpenAI

from src.embedding_model import embed
from src.utils import (
    populate_concerned_year,
    populate_everything,
    return_paragraphs_with_highest_score,
)

In [17]:
with open("questions.json", "r") as f:
    questions = json.load(f)

In [18]:
df: pd.DataFrame = pd.read_pickle("df_with_embeddings_float32_filtered50.pkl")
populate_concerned_year(df)

# filtering only years 2022
df = df[df["concerned_year"] == 2022]

In [19]:
import pickle
with open("questions_with_embeddings.pkl", "rb") as f:
    questions_embedding = pickle.load(f)

In [20]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"))
model = "gpt-4-turbo"
examples_to_retrieve = 15
for x, y in questions.items():
    if (
        (len(questions[x]["reference_texts"]) != 0)
        and (len(questions[x]["retrieved_texts"]) != 0)
        and (len(questions[x]["final_answer"]) != 0)
    ):
        print(f"Question {x} already answered, skipping...")
        continue
    print(f"Executing code for question {x}...")
    question_reference = y["question_reference"]
    question_final = y["question_final"]
    if len(questions[x]["reference_texts"]) != 0:
        print(f"Reference texts for question {x} already present, skipping...")
        inn_texts = y["reference_texts"]
    else:
        print(f"Getting reference texts from LLM for question:\n{
              question_reference}")
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "Jeg har en embedding-database med tekster fra offentlige dokumenter, og brukeren stiller spørsmål om tekstene i databasen. Gitt brukerens spørsmål, lag fem referansetekster som kan brukes som kan embeddes og sammenlignes med embeddingene i databasen med cosine-likhet for å finne tekster som kan brukes for å svare på brukerens spørsmål. Tekstene du lager skal være av varierende lengde slik at det fanger opp stikkordet som sees etter, i tillegg til noen lengre tekster som reflekterer ordlyden i tekstene som kan hjelpe å besvare spørsmålene. Pass på at tekstene er relativt ulike hverandre, slik at man kan ta gjennomsnittet av embeddingene for å fange opp essensen. Referansetekstene du lager skal ikke svare på spørsmålet til brukeren, men brukes for å finne tekster i databasen som ligner på tekster som kan besvare brukeren. Svaret ditt skal kun være en python liste, uten forklarende tekst",
                },
                {
                    "role": "user",
                    "content": f"{question_reference}",
                },
            ],
        )
        inn_texts = eval(completion.choices[0].message.content)
        y["reference_texts"] = inn_texts
    print("Embedding reference texts...")
    # inn_texts_embeddings = [embed(inn_text) for inn_text in inn_texts]
    # inn_texts_embeddings = np.array(inn_texts_embeddings)
    # inn_texts_embedding = np.mean(inn_texts_embeddings, axis=0)
    inn_texts_embedding = questions_embedding[x]["reference_embedding"]
    print("Comparing similarity...")
    # else include a series of true with the same length as the dataframe
    include_years = (
        df["type"] == "Årsrapport" if y["year_reps"] else np.array([
                                                                   False] * len(df))
    )
    include_grants = (
        df["type"] == "Tildelingsbrev" if y["grants"] else np.array([
                                                                    False] * len(df))
    )
    filtered_df = populate_everything(
        df[(include_years) | (include_grants)].copy(), inn_texts_embedding
    )
    if len(questions[x]["retrieved_texts"]) != 0:
        print(f"Retrieved texts for question {x} already present, skipping...")
        retrieved_texts = "\n\n".join(y["retrieved_texts"])
    else:
        retrieved_texts = return_paragraphs_with_highest_score(
            filtered_df, "deviation_scaled_with_length", examples_to_retrieve
        )
        y["retrieved_texts"] = retrieved_texts.strip().split("\n\n")
    if len(questions[x]["final_answer"]) != 0:
        print(f"Final answer for question {x} already present, skipping...")
    else:
        print(f"Getting final answer from LLM for question:\n{question_final}")
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": f"Basert på et sett med teksnutter fra rapporter fra ulike statlige virksomheter som brukeren gir vil jeg at du skal svare på spørsmålet: {question_final}. Pass på at du svarer med grunnlag i tekstsnuttene som brukeren gir, og at du svarer på spørsmålet oppgitt tidligere.",
                },
                {
                    "role": "user",
                    "content": f"{retrieved_texts}",
                },
            ],
        )
        y["final_answer"] = completion.choices[0].message.content

Question 1.1 already answered, skipping...
Question 1.2 already answered, skipping...
Question 1.3 already answered, skipping...
Question 1.4 already answered, skipping...
Question 1.5 already answered, skipping...
Question 1.6 already answered, skipping...
Question 2.1 already answered, skipping...
Question 2.2 already answered, skipping...
Question 2.3 already answered, skipping...
Question 2.4 already answered, skipping...
Executing code for question 3.1...
Reference texts for question 3.1 already present, skipping...
Embedding reference texts...
Comparing similarity...
Retrieved texts for question 3.1 already present, skipping...
Getting final answer from LLM for question:
Er det en sammenheng mellom virksomhetene som omtaler digitalisering og digital transformasjon nivået på innovasjonsaktivitet? Altså, er det en korrelasjon mellom virksomheter som er innovative og som er langt fremme med digitalisering?
Executing code for question 3.2...
Reference texts for question 3.2 already p

In [21]:
with open("answered_questions.json", "w") as f:
    json.dump(questions, f, ensure_ascii=False)